# State Pattern

## Intent
Allow an object to alter its behavior when its internal state changes. The object will appear to change its class.

## Problem
You have an object that behaves differently depending on its state:
- Many conditional statements based on state
- Difficult to add new states
- State transitions scattered across methods
- Violation of Single Responsibility Principle

**Real-world examples:**
- TCP connections (closed, listening, established)
- Document workflow (draft, review, published)
- Media players (playing, paused, stopped)
- Order processing (new, paid, shipped, delivered)

## When to Use
✅ **Use when:**
- Object behavior depends on its state
- Many conditional statements based on state
- State transitions are complex
- Need to add states without modifying existing code

❌ **Avoid when:**
- Simple state machine with few states
- State transitions are trivial
- States don't affect behavior significantly

## Pattern Structure
```
┌─────────┐          ┌───────┐
│ Context │─────────►│ State │
├─────────┤          ├───────┤
│ state   │          │handle()│
│request()│          └───────┘
└─────────┘               ▲
                    ┌─────┴─────┐
              ┌─────┴──┐   ┌────┴─────┐
              │StateA  │   │StateB    │
              ├────────┤   ├──────────┤
              │handle()│   │handle()  │
              └────────┘   └──────────┘
```

## Example 1: Document Workflow (Without State)

**Problem**: Complex conditionals for state-dependent behavior

In [ ]:
# WITHOUT State Pattern - Messy conditionals
class Document:
    def __init__(self):
        self.state = "draft"  # draft, review, published
        self.content = ""
    
    def edit(self, text):
        if self.state == "draft":
            self.content = text
            print(f"✏️  Edited: {text}")
        elif self.state == "review":
            print("❌ Cannot edit during review")
        elif self.state == "published":
            print("❌ Cannot edit published document")
    
    def submit(self):
        if self.state == "draft":
            self.state = "review"
            print("📤 Submitted for review")
        elif self.state == "review":
            print("❌ Already in review")
        elif self.state == "published":
            print("❌ Already published")
    
    def approve(self):
        if self.state == "draft":
            print("❌ Must submit first")
        elif self.state == "review":
            self.state = "published"
            print("✅ Approved and published")
        elif self.state == "published":
            print("❌ Already published")
    
    # Adding new states means modifying ALL these methods!

## Implementation: State Pattern

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

# Forward declaration
class Document:
    pass

# State interface
class DocumentState(ABC):
    """Abstract state for document."""
    
    @abstractmethod
    def edit(self, document: Document, text: str) -> None:
        pass
    
    @abstractmethod
    def submit(self, document: Document) -> None:
        pass
    
    @abstractmethod
    def approve(self, document: Document) -> None:
        pass
    
    @abstractmethod
    def reject(self, document: Document) -> None:
        pass


# Concrete states
class DraftState(DocumentState):
    """Document is in draft mode."""
    
    def edit(self, document: Document, text: str) -> None:
        document.content = text
        print(f"✏️  Edited: {text}")
    
    def submit(self, document: Document) -> None:
        print("📤 Submitted for review")
        document.state = ReviewState()
    
    def approve(self, document: Document) -> None:
        print("❌ Cannot approve draft. Submit first.")
    
    def reject(self, document: Document) -> None:
        print("❌ Cannot reject draft.")


class ReviewState(DocumentState):
    """Document is under review."""
    
    def edit(self, document: Document, text: str) -> None:
        print("❌ Cannot edit during review. Reject first.")
    
    def submit(self, document: Document) -> None:
        print("❌ Already submitted for review.")
    
    def approve(self, document: Document) -> None:
        print("✅ Approved! Publishing document...")
        document.state = PublishedState()
    
    def reject(self, document: Document) -> None:
        print("🔙 Rejected. Back to draft.")
        document.state = DraftState()


class PublishedState(DocumentState):
    """Document is published."""
    
    def edit(self, document: Document, text: str) -> None:
        print("❌ Cannot edit published document.")
    
    def submit(self, document: Document) -> None:
        print("❌ Already published.")
    
    def approve(self, document: Document) -> None:
        print("❌ Already published.")
    
    def reject(self, document: Document) -> None:
        print("❌ Cannot reject published document.")


# Context
class Document:
    """Document with state-dependent behavior."""
    
    def __init__(self, title: str):
        self.title = title
        self.content = ""
        self.state: DocumentState = DraftState()
    
    def edit(self, text: str) -> None:
        self.state.edit(self, text)
    
    def submit(self) -> None:
        self.state.submit(self)
    
    def approve(self) -> None:
        self.state.approve(self)
    
    def reject(self) -> None:
        self.state.reject(self)
    
    def get_state_name(self) -> str:
        return self.state.__class__.__name__.replace("State", "")


# Demo
print("=== Document Workflow with State Pattern ===")

doc = Document("Project Proposal")
print(f"\n📄 Document: {doc.title}")
print(f"   State: {doc.get_state_name()}\n")

# Draft state
doc.edit("Initial content")
doc.edit("Updated content")

print(f"\n   State: {doc.get_state_name()}\n")

# Try to approve without submitting
doc.approve()

# Submit for review
doc.submit()
print(f"\n   State: {doc.get_state_name()}\n")

# Try to edit during review
doc.edit("Try to edit")

# Reject and go back to draft
doc.reject()
print(f"\n   State: {doc.get_state_name()}\n")

# Edit again and resubmit
doc.edit("Final content")
doc.submit()
print(f"\n   State: {doc.get_state_name()}\n")

# Approve
doc.approve()
print(f"\n   State: {doc.get_state_name()}\n")

# Try to edit published
doc.edit("Try to edit published")

print("\n✅ State transitions handled cleanly!")

## Real-World Example: TCP Connection States

In [ ]:
# State interface
class ConnectionState(ABC):
    """Abstract TCP connection state."""
    
    @abstractmethod
    def open(self, connection: 'TCPConnection') -> None:
        pass
    
    @abstractmethod
    def close(self, connection: 'TCPConnection') -> None:
        pass
    
    @abstractmethod
    def send(self, connection: 'TCPConnection', data: str) -> None:
        pass


# Concrete states
class ClosedState(ConnectionState):
    """Connection is closed."""
    
    def open(self, connection: 'TCPConnection') -> None:
        print("🔓 Opening connection...")
        connection.state = ListeningState()
    
    def close(self, connection: 'TCPConnection') -> None:
        print("❌ Already closed")
    
    def send(self, connection: 'TCPConnection', data: str) -> None:
        print("❌ Cannot send data. Connection is closed.")


class ListeningState(ConnectionState):
    """Connection is listening for incoming connections."""
    
    def open(self, connection: 'TCPConnection') -> None:
        print("✅ Connection established!")
        connection.state = EstablishedState()
    
    def close(self, connection: 'TCPConnection') -> None:
        print("🔒 Closing connection...")
        connection.state = ClosedState()
    
    def send(self, connection: 'TCPConnection', data: str) -> None:
        print("❌ Cannot send data. Still listening.")


class EstablishedState(ConnectionState):
    """Connection is established and ready."""
    
    def open(self, connection: 'TCPConnection') -> None:
        print("❌ Connection already established")
    
    def close(self, connection: 'TCPConnection') -> None:
        print("🔒 Closing connection...")
        connection.state = ClosedState()
    
    def send(self, connection: 'TCPConnection', data: str) -> None:
        print(f"📤 Sending: {data}")


# Context
class TCPConnection:
    """TCP connection with state-dependent behavior."""
    
    def __init__(self, host: str, port: int):
        self.host = host
        self.port = port
        self.state: ConnectionState = ClosedState()
    
    def open(self) -> None:
        self.state.open(self)
    
    def close(self) -> None:
        self.state.close(self)
    
    def send(self, data: str) -> None:
        self.state.send(self, data)
    
    def get_state(self) -> str:
        return self.state.__class__.__name__.replace("State", "")


# Demo
print("\n=== TCP Connection State Machine ===")

conn = TCPConnection("example.com", 80)
print(f"\n🌐 Connection to {conn.host}:{conn.port}")
print(f"   State: {conn.get_state()}\n")

# Try to send while closed
conn.send("Hello")

# Open connection
conn.open()
print(f"   State: {conn.get_state()}\n")

# Try to send while listening
conn.send("Hello")

# Establish connection
conn.open()
print(f"   State: {conn.get_state()}\n")

# Send data
conn.send("GET / HTTP/1.1")
conn.send("Host: example.com")

# Close connection
print()
conn.close()
print(f"   State: {conn.get_state()}\n")

# Try to send after closed
conn.send("Goodbye")

## Real-World Example: Media Player

In [ ]:
# State interface
class PlayerState(ABC):
    """Abstract media player state."""
    
    @abstractmethod
    def play(self, player: 'MediaPlayer') -> None:
        pass
    
    @abstractmethod
    def pause(self, player: 'MediaPlayer') -> None:
        pass
    
    @abstractmethod
    def stop(self, player: 'MediaPlayer') -> None:
        pass


# Concrete states
class StoppedState(PlayerState):
    """Player is stopped."""
    
    def play(self, player: 'MediaPlayer') -> None:
        print(f"▶️  Playing: {player.current_track}")
        player.state = PlayingState()
    
    def pause(self, player: 'MediaPlayer') -> None:
        print("❌ Cannot pause. Not playing.")
    
    def stop(self, player: 'MediaPlayer') -> None:
        print("❌ Already stopped")


class PlayingState(PlayerState):
    """Player is playing."""
    
    def play(self, player: 'MediaPlayer') -> None:
        print("❌ Already playing")
    
    def pause(self, player: 'MediaPlayer') -> None:
        print(f"⏸️  Paused: {player.current_track}")
        player.state = PausedState()
    
    def stop(self, player: 'MediaPlayer') -> None:
        print(f"⏹️  Stopped: {player.current_track}")
        player.state = StoppedState()


class PausedState(PlayerState):
    """Player is paused."""
    
    def play(self, player: 'MediaPlayer') -> None:
        print(f"▶️  Resuming: {player.current_track}")
        player.state = PlayingState()
    
    def pause(self, player: 'MediaPlayer') -> None:
        print("❌ Already paused")
    
    def stop(self, player: 'MediaPlayer') -> None:
        print(f"⏹️  Stopped: {player.current_track}")
        player.state = StoppedState()


# Context
class MediaPlayer:
    """Media player with state-dependent behavior."""
    
    def __init__(self):
        self.state: PlayerState = StoppedState()
        self.current_track = "No track loaded"
    
    def load_track(self, track: str) -> None:
        self.current_track = track
        print(f"💿 Loaded: {track}")
    
    def play(self) -> None:
        self.state.play(self)
    
    def pause(self) -> None:
        self.state.pause(self)
    
    def stop(self) -> None:
        self.state.stop(self)
    
    def get_state(self) -> str:
        return self.state.__class__.__name__.replace("State", "")


# Demo
print("\n=== Media Player State Machine ===")

player = MediaPlayer()
player.load_track("Bohemian Rhapsody - Queen")
print(f"State: {player.get_state()}\n")

# Try to pause when stopped
player.pause()

# Play
player.play()
print(f"State: {player.get_state()}\n")

# Pause
player.pause()
print(f"State: {player.get_state()}\n")

# Resume
player.play()
print(f"State: {player.get_state()}\n")

# Stop
player.stop()
print(f"State: {player.get_state()}")

## Real-World Example: Order Processing

In [ ]:
# State interface
class OrderState(ABC):
    """Abstract order state."""
    
    @abstractmethod
    def pay(self, order: 'Order') -> None:
        pass
    
    @abstractmethod
    def ship(self, order: 'Order') -> None:
        pass
    
    @abstractmethod
    def deliver(self, order: 'Order') -> None:
        pass
    
    @abstractmethod
    def cancel(self, order: 'Order') -> None:
        pass


# Concrete states
class NewOrderState(OrderState):
    """Order is new, awaiting payment."""
    
    def pay(self, order: 'Order') -> None:
        print(f"💳 Payment received for order #{order.order_id}")
        order.state = PaidState()
    
    def ship(self, order: 'Order') -> None:
        print("❌ Cannot ship. Payment required.")
    
    def deliver(self, order: 'Order') -> None:
        print("❌ Cannot deliver. Not shipped yet.")
    
    def cancel(self, order: 'Order') -> None:
        print(f"🚫 Order #{order.order_id} cancelled")
        order.state = CancelledState()


class PaidState(OrderState):
    """Order is paid, ready to ship."""
    
    def pay(self, order: 'Order') -> None:
        print("❌ Already paid")
    
    def ship(self, order: 'Order') -> None:
        print(f"📦 Order #{order.order_id} shipped")
        order.state = ShippedState()
    
    def deliver(self, order: 'Order') -> None:
        print("❌ Cannot deliver. Not shipped yet.")
    
    def cancel(self, order: 'Order') -> None:
        print(f"🔄 Refunding order #{order.order_id}...")
        order.state = CancelledState()


class ShippedState(OrderState):
    """Order is shipped, in transit."""
    
    def pay(self, order: 'Order') -> None:
        print("❌ Already paid")
    
    def ship(self, order: 'Order') -> None:
        print("❌ Already shipped")
    
    def deliver(self, order: 'Order') -> None:
        print(f"✅ Order #{order.order_id} delivered!")
        order.state = DeliveredState()
    
    def cancel(self, order: 'Order') -> None:
        print("❌ Cannot cancel. Already in transit.")


class DeliveredState(OrderState):
    """Order is delivered."""
    
    def pay(self, order: 'Order') -> None:
        print("❌ Already paid")
    
    def ship(self, order: 'Order') -> None:
        print("❌ Already delivered")
    
    def deliver(self, order: 'Order') -> None:
        print("❌ Already delivered")
    
    def cancel(self, order: 'Order') -> None:
        print("❌ Cannot cancel delivered order. Request return instead.")


class CancelledState(OrderState):
    """Order is cancelled."""
    
    def pay(self, order: 'Order') -> None:
        print("❌ Order is cancelled")
    
    def ship(self, order: 'Order') -> None:
        print("❌ Order is cancelled")
    
    def deliver(self, order: 'Order') -> None:
        print("❌ Order is cancelled")
    
    def cancel(self, order: 'Order') -> None:
        print("❌ Already cancelled")


# Context
class Order:
    """Order with state-dependent behavior."""
    
    def __init__(self, order_id: int, items: list):
        self.order_id = order_id
        self.items = items
        self.state: OrderState = NewOrderState()
    
    def pay(self) -> None:
        self.state.pay(self)
    
    def ship(self) -> None:
        self.state.ship(self)
    
    def deliver(self) -> None:
        self.state.deliver(self)
    
    def cancel(self) -> None:
        self.state.cancel(self)
    
    def get_state(self) -> str:
        return self.state.__class__.__name__.replace("State", "")


# Demo
print("\n=== Order Processing State Machine ===")

# Successful order
print("\n--- Order 1: Success Flow ---")
order1 = Order(1001, ["Laptop", "Mouse"])
print(f"📋 Order #{order1.order_id} created")
print(f"   Items: {', '.join(order1.items)}")
print(f"   State: {order1.get_state()}\n")

order1.pay()
print(f"   State: {order1.get_state()}\n")

order1.ship()
print(f"   State: {order1.get_state()}\n")

order1.deliver()
print(f"   State: {order1.get_state()}")

# Cancelled order
print("\n--- Order 2: Cancellation Flow ---")
order2 = Order(1002, ["Keyboard"])
print(f"📋 Order #{order2.order_id} created")
print(f"   State: {order2.get_state()}\n")

order2.pay()
print(f"   State: {order2.get_state()}\n")

order2.cancel()
print(f"   State: {order2.get_state()}\n")

# Try to ship cancelled order
order2.ship()

# Invalid transitions
print("\n--- Order 3: Invalid Transitions ---")
order3 = Order(1003, ["Monitor"])
print(f"📋 Order #{order3.order_id} created\n")

# Try to ship without paying
order3.ship()

# Try to deliver without shipping
order3.deliver()

## State vs Strategy Pattern

**State Pattern**:
- Object changes behavior based on its state
- State transitions happen internally
- States know about each other

**Strategy Pattern**:
- Object uses different algorithms
- Strategy is set externally
- Strategies are independent

In [ ]:
# State Pattern - States transition internally
class TrafficLight:
    def __init__(self):
        self.state = RedState()
    
    def change(self):
        self.state.change(self)  # State decides next state

class RedState:
    def change(self, light):
        print("Red -> Green")
        light.state = GreenState()  # Knows about next state

class GreenState:
    def change(self, light):
        print("Green -> Yellow")
        light.state = YellowState()

class YellowState:
    def change(self, light):
        print("Yellow -> Red")
        light.state = RedState()

# Demo
print("State Pattern (internal transitions):")
light = TrafficLight()
light.change()
light.change()
light.change()

# Strategy Pattern - Client sets strategy
class Compressor:
    def __init__(self, strategy):
        self.strategy = strategy  # Set externally
    
    def compress(self, data):
        return self.strategy.compress(data)

class ZipStrategy:
    def compress(self, data):
        return f"ZIP: {data}"

class RarStrategy:
    def compress(self, data):
        return f"RAR: {data}"

# Demo
print("\nStrategy Pattern (external selection):")
comp = Compressor(ZipStrategy())
print(comp.compress("file.txt"))

comp.strategy = RarStrategy()  # Changed externally
print(comp.compress("file.txt"))

## Advantages & Disadvantages

### ✅ Advantages
1. **Single Responsibility**: Each state is a separate class
2. **Open/Closed Principle**: Add new states without modifying context
3. **Eliminates conditionals**: No complex if/else chains
4. **Clear state transitions**: Explicit state change logic
5. **Easy to understand**: State behavior is localized

### ❌ Disadvantages
1. **More classes**: One class per state
2. **Complexity**: May be overkill for simple state machines
3. **State dependencies**: States may need to know about each other

## Common Use Cases

1. **Network connections**: TCP states (closed, listening, established)
2. **Document workflows**: Draft, review, published
3. **Media players**: Playing, paused, stopped
4. **Order processing**: New, paid, shipped, delivered
5. **Game characters**: Idle, running, jumping, attacking
6. **UI components**: Enabled, disabled, focused
7. **Vending machines**: Idle, selecting, dispensing

## Related Patterns

- **Strategy**: Similar structure, different intent (algorithm selection vs state)
- **Flyweight**: Can share state objects
- **Singleton**: Often used for state objects
- **Command**: Commands can trigger state transitions

## Best Practices

1. **Keep states simple**: Each state should have clear responsibilities
2. **Use state factory**: For creating state instances
3. **Consider singleton**: If states have no instance-specific data
4. **Document transitions**: Make state diagram/table
5. **Guard conditions**: Validate state transitions
6. **Entry/exit actions**: Add setup/cleanup methods if needed

## Summary

State pattern enables:
- State-dependent behavior
- Clean state transitions
- Elimination of conditionals
- Easy addition of new states

Perfect for: Workflows, connections, players, order processing, game states.

**Key Insight**: Encapsulate state-specific behavior in separate classes, making behavior changes explicit and maintainable!